In [ ]:
import os
import pickle
import json
import glob
from tqdm.auto import trange, tqdm
import numpy as np
from features import prepare_entry

In [2]:
DATA_DIR = 'data/small/'
GROUND_TRUTH_PATH = DATA_DIR + 'pan20-authorship-verification-training-small-truth.jsonl'
DATA_PATH = DATA_DIR + 'pan20-authorship-verification-training-small.jsonl'
TEMP_DATA_PATH = 'temp_data/small_model_training_data/'

In [ ]:
train_ids, test_ids, _, _ = pickle.load(open(TEMP_DATA_PATH + 'dataset_partition.p', 'rb'))

Code to run this locally
===

In [ ]:

with open(DATA_PATH, 'r') as f, \
    open(TEMP_DATA_PATH + 'preprocessed_train.jsonl', 'w') as f_train, \
    open(TEMP_DATA_PATH + 'preprocessed_test.jsonl', 'w') as f_test:
    for l in tqdm(f, total=len(train_ids) + len(test_ids)):
        d = json.loads(l)
        preprocessed = {
            'id': d['id'],
            'fandoms': d['fandoms'],
            'pair': [
                prepare_entry(d['pair'][0], mode='accurate', tokenizer='casual'),
                prepare_entry(d['pair'][1], mode='accurate', tokenizer='casual')
            ]
        }
        if d['id'] in train_ids:
            json.dump(preprocessed, f_train)
            f_train.write('\n')
        else:
            json.dump(preprocessed, f_test)
            f_test.write('\n')

Merge HPC-Preprocessed Files
===

Only run this if the preprocessing was done in the HPC

In [3]:
from preprocess_hpc import NUM_MACHINES

In [4]:
with open(TEMP_DATA_PATH + 'preprocessed_train.jsonl', 'w') as f_train:
    for i in trange(NUM_MACHINES):
        with open(TEMP_DATA_PATH + 'preprocessed_train_' + str(i) + '.jsonl', 'r') as f:
            for l in f:
                _ = f_train.write(l)

In [5]:
with open(TEMP_DATA_PATH + 'preprocessed_test.jsonl', 'w') as f_test:
    for i in trange(NUM_MACHINES):
        with open(TEMP_DATA_PATH + 'preprocessed_test_' + str(i) + '.jsonl', 'r') as f:
            for l in f:
                _ = f_test.write(l)